# SageMaker で提供される Linear Learner アルゴリズムの学習と推論を行う

#### ノートブックに含まれる内容

- Amazon が提供するアルゴリズムの使いかた
- 中でも，Linear Learner (線形回帰) アルゴリズムの使い方

#### ノートブックで使われている手法の詳細

- アルゴリズム: Linear Learner
- データ: abalone データ


## セットアップ

In [ ]:
import boto3
import re

import os
import csv
import json
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

# AWS credential で指定された role を返す
role = get_execution_role()

In [ ]:
# SageMaker のセッションを作成
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## データのロード

SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，データを SageMaker SDK が用意しているメソッドを用いて，ラベルデータと特徴量データに分けた上で，RecordIO フォーマットに変換します．その上で S3 にアップロードする形をとります．SageMaker の LinearLearner では

ここではいったん，numpy 形式でデータを読み込み，ラベルデータと特徴量データに分割します．また，Linear Learner ではデータ型として float32 のみをサポートしているため，ここでは numpy array として読み込んだデータを float32 にキャストしています．

In [ ]:
train_data = np.loadtxt('data/abalone_train.csv', delimiter=',', dtype='float32')
train_labels = train_data[:, 0]
train_features = train_data[:, 1:]

## モデルの学習を実行

データの準備ができたら，さっそく学習ジョブを実行しましょう．ここでは，SageMaker SDK であらかじめ用意されている `LinearLearner` クラスを用います．`LinearLearner` オブジェクトを作成したら，`record_set()` メソッドを用いて RecordIO 形式に変換したデータを S3 にアップした上で，このデータを用いて `fit()` メソッドで学習ジョブを走らせます．

In [ ]:
import boto3
from sagemaker import LinearLearner, LinearLearnerModel

linear_estimator = LinearLearner(role, 
                                 train_instance_count=1, 
                                 train_instance_type='ml.m4.xlarge',
                                 predictor_type='regressor')

records = linear_estimator.record_set(train_features, labels=train_labels)

linear_estimator.fit(records)

## モデルの推論を実行

推論を行うために，Estimateor オブジェクトからモデルオブジェクトを作成した上で，モデルをエンドポイントにデプロイします．deploy() メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
linear_model = LinearLearnerModel(linear_estimator.model_data,
                                  role=role,
                                  sagemaker_session=sess)

In [ ]:
linear_predictor = linear_model.deploy(initial_instance_count=1,
                                       instance_type='ml.m4.xlarge')

エンドポイントの準備ができたら，boto3 クライアント経由で，ローカルファイルから読み込んだテストデータをエンドポイントに投げて，予測を行い結果を表示してみます．

In [ ]:
import boto3

sagemaker = boto3.client('sagemaker-runtime')

with open('data/abalone_predict.csv') as f:
    csvreader = csv.reader(f)
    for line in csvreader:
        label = line[0]
        features = ','.join(line[1:])
        response = sagemaker.invoke_endpoint(
            EndpointName=linear_predictor.endpoint,
            Body=features,
            ContentType='text/csv'
        )
        predicted = json.loads(response["Body"].read())['predictions'][0]['score']
        print('correct label: {:2}, predicted label: {:>2.2f}'.format(int(label), float(predicted)))

In [ ]:
with open('data/abalone_predict.csv') as f:
    csvreader = csv.reader(f)
    for line in csvreader:
        label = line[0]
        features = ','.join(line[1:])
    print(features)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(linear_predictor.endpoint)